# Unprocessed dwelling information extractor
This notebook extracts usefull NaN information per dwelling and saves this into one file (csv & excel.  
It will this for both the 10s and hour sample rate dataframes.  
The unprocessed dataframes are loaded from: `//datc//opschaler//combined_gas_smart_weather_dfs//unprocessed//`  
The nan information per dwelling is loaded from: `//datc//opschaler//nan_information//`  
The final product is saved in: `//datc//opschaler//dwelling_information//total_information//`  
It contains the following information per dwelling, for both the 10s and one hour sample rate:  
* dwelling id  
Dwelling id of the house.
  
* recorded days  
The length of the unprocessed dataframe in days.
  
* total samples (per columnm in thousands)  
Total amount of samples in thousands, per column.
  
* total NaN streaks  
Total amount of NaN streaks.
  
* total NaN streaks > 2  
Total amount of NaN streaks which are larger than 2.
  
* total NaNs [-]  
Total amount of NaNs in the unprocessed dataframe.
  
* total NaNs [%]  
Totals NaNs devided by the total samples.
  
* mean of NaNs  
Mean of the amount of NaNs per NaN streak.
  
* median of NaNs  
Median of the amount of NaNs per NaN streak.
  
* std of NaNs  
Standard deviation of the amount of NaNs per NaN streak.
  
* first highest NaN streak (%)  
Amount of NaNs from the first highest NaN streak, devided by the total samples.
  
* first highest NaN streak column  
The name of the column where the first highest NaN streak is in.
  
* second highest NaN streak (%)  
Amount of NaNs from the first highest NaN streak, devided by the total samples.
  
* second highest NaN streak column  
The name of the column where the first highest NaN streak is in.
  
* third highest NaN streak (%)  
Amount of NaNs from the first highest NaN streak, devided by the total samples.
  
* third highest NaN streak column  
The name of the column where the first highest NaN streak is in.
  

# Imports

In [8]:
import glob
import pandas as pd
import numpy as np

# Function definitions

In [9]:
def unprocessed_length_in_days(dwelling_id, type_):
    """
    Get the total amount of days of the unprocessed dwelling_id.
    """
    dir = '//datc//opschaler//combined_gas_smart_weather_dfs//unprocessed//'
    df = pd.read_csv(dir+dwelling_id+'_'+type_+'.csv', delimiter='\t', parse_dates=['datetime'])
    columns = df.columns
    df = df['datetime'] # only keep the datetime column
    start_date = df[0]
    stop_date = df[-1:]
    
    del df # Free up memory
    recorded_days = (stop_date - start_date).dt.days # total amount of recorded days
    recorded_days = recorded_days.reset_index() # Messy way to convert the Series to a DataFrame
    recorded_days = recorded_days['datetime'][0]

    return recorded_days, columns


def nan_information_extractor(dwelling_id, path, type_):
    """
    Extracts usefull information from the nan info table from a dwelling id. 
    Output is a list with this information.
    """
    df = pd.read_csv(path, delimiter='\t')
    df = df.sort_values(by=['Amount of NaNs'], ascending=False) # Sort from highest to lowest amount of NaNs
    
    recorded_days, columns = unprocessed_length_in_days(dwelling_id, type_) # Length of unprocessed dataframe in days
    
    if df.empty: # If df is empty, return nothins
        print('Dataframe is empty: %s' % path)
        result = list(np.full(15, np.NaN)) # Make all outputs NaN
        result[0] = dwelling_id
        result[1] = recorded_days
        return result
    else:

        if type_ == 'hour':
            length = recorded_days*24
        elif type_ == '10s':
            length = recorded_days*24*60*6
        else: 
            print('type_ must be \'hour\' or \'10s')
    
        # Calculate usefull information
        #total_samples = length*len(columns) # get the total amount of samples in the complete df
        total_samples = length
        total_gaps = len(df['Amount of NaNs'])
        total_gaps_larger_than_2 = len(df[df['Amount of NaNs'] > 2])
        total_nans = df['Amount of NaNs'].sum()
        total_nans_percentage = (total_nans / total_samples)*100
        mean = df['Amount of NaNs'].mean()
        median = df['Amount of NaNs'].median()
        std = df['Amount of NaNs'].std()
        
        # Try to get relevant values for the top 3 of NaN streaks
        # Problem with this is that often there are multiple columns which have the same NaN streak...
        try: 
            first_highest_p = (df['Amount of NaNs'][0]/ total_samples)*100
            first_highest_column = df['Column name'][0]
        except:
            print('There is no 1st highest')
            first_highest_p = np.NaN
            first_highest_column = np.NaN
        
        try:
            second_highest_p = (df['Amount of NaNs'][1]/ total_samples)*100
            second_highest_column = df['Column name'][1]
        except:
            print('There is no 2nd highest')
            second_highest_p = np.NaN
            second_highest_column = np.NaN
        
        try:
            third_highest_p = (df['Amount of NaNs'][3]/ total_samples)*100
            third_highest_column = df['Column name'][3]
        except:
            print('There is no 3rd highest')
            third_highest_p = np.NaN
            third_highest_column = np.NaN
        
    
        # Put the results in a list
        result = [dwelling_id, recorded_days, total_samples, total_gaps, total_gaps_larger_than_2, total_nans, total_nans_percentage, mean, median, 
                  std, first_highest_p, first_highest_column, second_highest_p, second_highest_column, third_highest_p, third_highest_column]
        
        return result

# Main

In [10]:
def main():
    nan_dir = '//datc//opschaler//nan_information//'
    paths_h = glob.glob(nan_dir+'*_hour.csv')
    ids_h = list(map(lambda x: x[-20:-9], paths_h))
    
    paths_s = glob.glob(nan_dir+'*_hour.csv')
    ids_s = list(map(lambda x: x[-20:-9], paths_h))
    
    results_h = []
    results_10s = []
    
    headers=['dwelling id', 'recorded days', 'total samples (per column, in thousands)','total NaN streaks', 'total NaN streaks > 2','total NaNs [-]', 'total NaNs [%]', 'mean of NaNs', 'median of NaNs', 'std of NaNs', 
             'first highest NaN streak (%)', 'first highest NaN streak column', 
             'second highest NaN streak (%)', 'second highest NaN streak column', 
             'third highest NaN streak (%)', 'third highest NaN streak column']
    
    for i, path in enumerate(paths_h):
        dwelling_id = ids_h[i]
        type_ = 'hour'
        results_h.append(nan_information_extractor(dwelling_id, path, type_))
    
    for i, path in enumerate(paths_s):
        print('10s at %s of %s' % (i, len(paths_s)))
        dwelling_id = ids_s[i]
        type_ = '10s'
        results_10s.append(nan_information_extractor(dwelling_id, path, type_))
        
    # make df from list of lists, round all values within to 1 decimal.
    df_hour = pd.DataFrame.from_records(results_h, columns=headers)
    # round some numbers
    
    df_10s = pd.DataFrame.from_records(results_10s, columns=headers)
    
    # sort by recoded days, highest to lowest
    df_hour = df_hour.sort_values(by=['recorded days'], ascending=False) 
    df_10s = df_10s.sort_values(by=['recorded days'], ascending=False) 
    
    return df_10s, df_hour

# Run main() and save the result

In [ ]:
%time info_10s, info_hour = main() # This takes ~2,5 minutes

83 24
145 24
132 24
84 24
83 24
1 24
85 24
85 24
74 24
83 24
39 24
Dataframe is empty: //datc//opschaler//nan_information/P01S01W7042_hour.csv
86 24
83 24
147 24
84 24
27 24
Dataframe is empty: //datc//opschaler//nan_information/P01S01W1554_hour.csv
76 24
87 24
220 24
211 24
85 24
83 24
83 24
128 24
96 24
210 24
23 24
26 24
There is no 3rd highest
82 24
98 24
155 24
92 24
22 24
141 24
76 24
83 24
83 24
84 24
84 24
83 24
83 24
83 24
146 24
94 24
26 24
86 24
85 24
212 24
147 24
86 24
84 24
0 24
Dataframe is empty: //datc//opschaler//nan_information/P01S01W0001_hour.csv
83 24
85 24
85 24
152 24
10s at 0 of 56
83 24
10s at 1 of 56
145 24
10s at 2 of 56
132 24
10s at 3 of 56
84 24
10s at 4 of 56
83 24
10s at 5 of 56
1 24
10s at 6 of 56
85 24
10s at 7 of 56
85 24
10s at 8 of 56
74 24
10s at 9 of 56
83 24
10s at 10 of 56
39 24
Dataframe is empty: //datc//opschaler//nan_information/P01S01W7042_hour.csv
10s at 11 of 56
86 24
10s at 12 of 56
83 24
10s at 13 of 56
147 24
10s at 14 of 56
84 24
10s

In [ ]:
info_hour

info_hour.to_csv('//datc//opschaler//dwelling_information//total_information//total_nan_information_hour.csv', sep='\t', index=False)
info_10s.to_csv('//datc//opschaler//dwelling_information//total_information//nan_information_10s.csv', sep='\t', index=False)

# Also save to Excel
writer = pd.ExcelWriter('//datc//opschaler//dwelling_information//total_information//total_nan_information.xlsx')
info_hour.to_excel(writer,'Hour dataframes', index=False)
info_10s.to_excel(writer,'10s dataframes', index=False)
writer.save()

print('FINISHED')